In [1]:
# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

- [07 PyTorch에서 다양한 장치 간 모델을 저장하고 불러오기](https://tutorials.pytorch.kr/recipes/recipes/save_load_across_devices.html)
- 